In [14]:
import pandas as pd
import matplotlib.pyplot as plt
from read_data import get_dirs
import os

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image

In [8]:
class PhotoDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row["path"]).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        label = torch.tensor(row["label"], dtype=torch.long)
        return image, label


In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


In [15]:
df = get_dirs()

dataset = PhotoDataset(df, transform=transform)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

images, labels = next(iter(train_loader))
print(images.shape, labels.shape)



torch.Size([16, 3, 224, 224]) torch.Size([16])


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using:", device)

model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 2)   # Alex vs Kelly
model = model.to(device)

Using: cpu
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/sumanth/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100.0%


In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [19]:
model.train()

for batch_idx, (imgs, lbls) in enumerate(train_loader):
    imgs, lbls = imgs.to(device), lbls.to(device)

    optimizer.zero_grad()
    outputs = model(imgs)
    loss = criterion(outputs, lbls)
    loss.backward()
    optimizer.step()

    print(f"Batch {batch_idx}: Loss = {loss.item():.4f}")

    if batch_idx == 5:
        break


Batch 0: Loss = 0.7322
Batch 1: Loss = 0.6119
Batch 2: Loss = 0.5328
Batch 3: Loss = 0.6830
Batch 4: Loss = 0.4426
Batch 5: Loss = 0.5724
